In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 26 17:17:34 2020

"""
import numpy as np
import scipy
from scipy import interpolate
from scipy import optimize, io, linalg
import os
import xarray as xr
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d
from scipy.interpolate import interp2d
from scipy.interpolate import griddata

/home/gabin/anaconda3/lib/python3.7/site-packages/distributed/utils.py:134: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  % (host, default, e), RuntimeWarning)


## import data
path_nadir and path_SWOTpass are imported on the OSMOSIS1 domain:

In [3]:
#%%
#### SWOT DATA
# Make sure to put this script in the directory below (/data/OSMOSIS/) before you run it
domain_name = 'OSMOSIS1'

# Name of the SWOT pass in data/*domain*/ directory
if domain_name == 'OSMOSIS1' :
    path_SWOTpass = 'BOOST-SWOT_SWOT_OSMOSIS_c02_p436.nc'
    path_nadir = 'BOOST-SWOT_J1_nadir_c05_p01.nc'
    # Plotting params
    max_range = 0.2 # Limit for colorbar (SSH in meters) for SSH
    max_range_diff = 0.1 # Limit for colorbar (SSH in meters) for SSH differences

Then a SWOT path is donwload

In [4]:
#### Download a SWOT path
with xr.open_dataset(path_SWOTpass, mask_and_scale=True) as (ds_SWOTpass):
    ds_SWOTpass.info() # Dimension of the dataset
ntime = ds_SWOTpass.time.size
nc = ds_SWOTpass.nC.size
lon_swotpass = np.zeros([ntime, nc])
lon_swotpass [:,:] = ds_SWOTpass.lon
lon_swotpass = np.ma.masked_invalid(lon_swotpass)
lat_swotpass = np.zeros([ntime,nc])
lat_swotpass [:,:] = ds_SWOTpass.lat
lat_swotpass = np.ma.masked_invalid(lat_swotpass)
ssh_swotpass = np.zeros([ntime,nc])
if roll_error_only:
    ssh_swotpass[:, :] = ds_SWOTpass.ssh_model + ds_SWOTpass.roll_err
elif all_but_karin:
   ssh_swotpass[:, :] = ds_SWOTpass.ssh_model + ds_SWOTpass.roll_err + ds_SWOTpass.phase_err + ds_SWOTpass.timing_err + ds_SWOTpass.bd_err
else: 
    ssh_swotpass[:, :] = ds_SWOTpass.ssh_obs
ssh_swotpass = np.ma.masked_invalid(ssh_swotpass)
ssh_swotpass_truth = np.zeros([ntime, nc]) 
ssh_swotpass_truth[:, :] = ds_SWOTpass.ssh_model # ssh model
ssh_swotpass_truth = np.ma.masked_invalid(ssh_swotpass_truth)



ValueError: numpy.ufunc size changed, may indicate binary incompatibility. Expected 124 from C header, got 112 from PyObject

In [5]:
#### Download NADIR DATA
with xr.open_dataset(path_nadir, mask_and_scale=True) as ds_nadirpass:
    ds_nadirpass.info()
ndtime = ds_nadirpass.time.size
lon_nadirpass = np.zeros([ndtime])
lon_nadirpass[:] = ds_nadirpass.lon
lat_nadirpass = np.zeros([ndtime])
lat_nadirpass[:] = ds_nadirpass.lat
x_al = np.zeros([ndtime])
x_al[:] = ds_nadirpass.x_al
ssh_nadir = np.zeros([ndtime])  
            
ssh_nadir[:] = ds_nadirpass.ssh_obs
ssh_nadir_truth = np.zeros([ndtime])  
ssh_nadir_truth[:] = ds_nadirpass.ssh_model


AttributeError: type object 'netCDF4._netCDF4._MemBuf' has no attribute '__reduce_cython__'

In [6]:
### SSH vs Nadir along track (x_al) for Nadir data
plt.figure()
#plt.scatter(lon_swotpass, lat_swotpass, c=ssh_swotpass, s=15)
plt.plot(x_al,ssh_nadir)
plt.ylabel('SSH (m)')
plt.xlabel('x_al (km)')
plt.title('Nadir data along track', fontsize=15)
#cb = plt.colorbar()
#cb.ax.tick_params(labelsize=15)

NameError: name 'x_al' is not defined

<Figure size 432x288 with 0 Axes>

In [7]:
### select SSH data along the track (x_al)
def SWOT_along_x_al (lon_nadirpass,lat_nadirpass,lon_swotpass,lat_swotpass,ssh_swotpass,ndtime,ntime,nc):
    ssh_swotxal=np.zeros[ndtime]
    for i in range (0,ntime):
        for j in range (0,nc):
            for k in range (0,ndtime):
                if lon_nadirpass[k]==lon_swotpass[i,j] and lat_nadirpass[k]==lat_swotpass[i,j]:
                    ssh_swotxal[k]=ssh_swotpass[i,j]
    return (ssh_swotxal)
    

In [8]:
#%%
## To interpolate SWOT data into the Nadir line
linear_interp = interp1d(ssh_swotpass,ssh_nadir_truth, kind='cubic') # 
#linear_interp = interp2d(lon_swotpass,ssh_swotpass,ssh_nadir, kind='cubic')
#%%
####Another way
grid_x, grid_y = np.mgrid[0:100:100j, 0:100:200j]
grid_z0 = griddata(ssh_swotpass, ssh_nadir, (grid_x, grid_y), method='linear')

NameError: name 'ssh_swotpass' is not defined